## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
# from denoiser import Denoiser

In [3]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

#### Setup hparams

In [4]:
hparams = create_hparams()
hparams.sampling_rate = 22050

attention_dim: 128
attention_location_kernel_size: 31
attention_location_n_filters: 32
attention_rnn_dim: 1024
batch_size: 16
cudnn_benchmark: false
cudnn_enabled: true
decoder_rnn_dim: 1024
dist_backend: nccl
dist_url: tcp://localhost:54321
distributed_run: false
dynamic_loss_scaling: true
encoder_embedding_dim: 512
encoder_kernel_size: 5
encoder_n_convolutions: 3
epochs: 37
filter_length: 1024
fp16_run: false
gate_threshold: 0.5
grad_clip_thresh: 1.0
hop_length: 256
ignore_layers:
- embedding.weight
iters_per_checkpoint: 200
learning_rate: 0.001
load_mel_from_disk: false
mask_padding: true
max_decoder_steps: 1000
max_wav_value: 32768.0
mel_fmax: 8000.0
mel_fmin: 0.0
n_frames_per_step: 1
n_mel_channels: 80
n_symbols: 150
p_attention_dropout: 0.1
p_decoder_dropout: 0.1
postnet_embedding_dim: 512
postnet_kernel_size: 5
postnet_n_convolutions: 5
prenet_dim: 256
sampling_rate: 22050
seed: 1234
symbols_embedding_dim: 512
text_cleaners:
- flowtron_cleaners
training_files: filelists/vlsp/tra

#### Load model from checkpoint

In [5]:
checkpoint_path = "./checkpoints/checkpoint_10400"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [6]:
# waveglow_path = 'waveglow_256channels.pt'
# waveglow = torch.load(waveglow_path)['model']
# waveglow.cuda().eval().half()
# for k in waveglow.convinv:
#     k.float()
# denoiser = Denoiser(waveglow)

#### Prepare text input

In [ ]:
text = "nếu bây giờ mà phá một chiếc cầu trong vụ này . chúng nó sẽ lùng chúng tôi"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [9]:
import random
import numpy as np
import torch
import torch.utils.data

import layers
from utils import load_wav_to_torch, load_filepaths_and_text
from text import text_to_sequence
from text import _clean_text, get_arpabet
from text import cmudict
import re
from text import text_embedding
text = "nếu bây giờ mà phá một chiếc cầu trong vụ này . chúng nó sẽ lùng chúng tôi"
text = _clean_text(text, hparams.text_cleaners)
words = re.findall(r'\S*\{.*?\}\S*|\S+', text)

cmudictx = cmudict.CMUDict(
            './text/phone_vn_north.txt',
            keep_ambiguous=True
        )

text = ' '.join([get_arpabet(word,cmudictx)
                 if random.random() < 0.5 
                 else word
                 for word in words])

In [ ]:
text_norm =self.text_embedding.text_norm(text)# text_embedding.text_norm(text)
# text_out =self.text_embedding.g2s_model(text_norm)  #text_embedding.g2s(text_norm)
# print(text_norm)
sequence = self.text_embedding.text2seq(text_norm)
# text_norm = torch.IntTensor(text_to_sequence(text, self.text_cleaners))
text_norm = torch.LongTensor(sequence)

In [11]:
from data_utils import TextMelLoader, TextMelCollate
trainset = TextMelLoader(hparams.training_files, hparams)

In [16]:
inputs=trainset.get_text(
    text
)
sequence = torch.autograd.Variable(
    inputs.unsqueeze(0)).cuda().long()
sequence.size()

torch.Size([1, 76])

In [17]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

In [20]:
mel_re =torch.exp(mel_outputs_postnet)
mel_re

tensor([[[1.4770e-04, 6.4898e-04, 5.8556e-03,  ..., 1.1330e-02,
          9.2850e-03, 4.3831e-03],
         [9.1016e-05, 2.8467e-04, 6.2561e-03,  ..., 1.1826e-02,
          8.8882e-03, 3.7365e-03],
         [5.6922e-05, 2.3782e-04, 8.5144e-03,  ..., 8.7204e-03,
          6.1111e-03, 2.7866e-03],
         ...,
         [1.5557e-05, 2.7955e-05, 1.7405e-04,  ..., 2.8920e-04,
          1.9717e-04, 1.4424e-04],
         [1.4424e-05, 2.6464e-05, 1.7953e-04,  ..., 2.5725e-04,
          1.7536e-04, 1.2636e-04],
         [1.4961e-05, 2.6464e-05, 1.8096e-04,  ..., 2.7156e-04,
          1.8668e-04, 1.3137e-04]]], device='cuda:0', dtype=torch.float16,
       grad_fn=<ExpBackward>)

In [25]:
mel_re.size()

torch.Size([1, 80, 386])

In [31]:
import librosa
S = librosa.feature.inverse.mel_to_stft(mel_re.squeeze(0).detach().cpu().numpy(),
                                        n_fft = 1024)
#                                         hop_length=25)

TypeError: array type float16 is unsupported in linalg

In [21]:
from audio_processing import griffin_lim

In [26]:
wavs = griffin_lim(mel_re.squeeze(0).detach().cpu(), trainset.stft.stft_fn)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [1026, 1, 1024], but got 2-dimensional input of size [80, 772] instead

#### Decode text input and plot results

In [ ]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [ ]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 